# WS2812 Overlay example

## Background
The overlay contains 64 hard WS2812 controllers (implemented in Zynq PL) connected to pins 0-63 on the PYNQ-Z1. See Appendix for pin mappings.

Each WS2812 can control an LED string up to a maximum of 2048 LEDs.

Each controller contains a dual-port BRAM at the address range 0 - 0x1fff. One port of the BRAM is connected to the Zynq PS, and is accessible from Python. The other BRAM port is connected to the WS2812 controller.  

<img src="images/ws2812_block_diagram.jpg">

The number of LEDs should be set by writing to a register in the IP at address 0x2000. 

From Python, LEDs can be set by writing a 24 bit value RGB value to the WS2812 BRAM (address 0, up to 2047 *4). Each 24 bit LED value is mapped to a 32 bit memory location in the IP.

---

## Current example
This example uses one 16 LED string.
https://www.adafruit.com/product/1463

Instantiate the ws2812 controller (automatically loads the overlay bitstream)

In [ ]:
from ws2812 import ws2812
ws2812 = ws2812()

## Get the IP address from the IP overlay 
print *ip* to see the full list of IP in the overlay

In [69]:
from pynq import Overlay
ol = Overlay(ws2812.bitfile)
ip = ol.ip_dict
address_1 = ip['SEG_ws2812_0_reg0'][0]

## Write to the LEDs
A simple write_led function is used to write an LED value to the array. Sliders are also used to select the individual LED, and to also select the R, G, B values

In [67]:
import ipywidgets as widgets
from ipywidgets import interactive
from IPython.display import display
from pynq import MMIO

NUMBER_LEDS_ADDR = 4096

address_1 = int(address_1, 16)
a = MMIO(address_1, 0x2000)
a.write(NUMBER_LEDS_ADDR, 16)

def write_led(pixel, red, blue, green):
    a.write(pixel * 4, (green << 16) + (red << 8) + (blue))
    print("Colour: " + hex((green << 16) + (red << 8) + (blue)))


color_sliders = interactive(write_led, blue=(0, 0xff, 1), red=(0, 0xff, 1), green=(0, 0xff, 1), pixel=(0, 15, 1))

display(color_sliders)


Colour: 0x7f7f7f


# Appendix
## LED BRAM limit
Number of LEDS limimted by BRAM size. Z7020 has 140 BRAM. This allows 2x BRAM for each controller (128 used in total).


## Pin mappings
Pins 0-63 as inndicated are connected to SEG_ws2812_0_reg0 to SEG_ws2812_63_reg0
<img = src="images/pin_diagram.jpg">